</div>
<div align="center">
  <img src="img/evolnode.png" width="800" alt="Fourier reconstruction convergence">
  <p><em> Evolution Node free you from coding and debugging, let LLM evolve your code for you.</em></p>
</div>
<div align="left">
<p><em>Bored of manual coding, a function? EvolNode let LLM automate function design and guide the evolution of it with genetic algorithm. A node here takes a task, input, and output, it uses either code or another LLM to complete the task, ensuring aligned input and output value types and ,.names.Fitness evaluation is done by running the function with a few specified test cases, the more diverse the test case, the better the evolution.
</em></p>
</div>

In [1]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode


# Code + Compilor Task
# mp = MetaPrompt("Generate Fibonacci sequence.", "fibonacci", ["n"], ["sequence"], ["int"], ["list"], PromptMode.CODE) # 
# node = EvolNode(mp, None, None)
# input_dict = {"n": 10}
# reasoning, code = node.evolve([input_dict], "i1", replace=True) # Evolution with guaranteed structural fitness
# node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 
node = EvolNode(mp, None, None)
input_dict = {"name": "Dilireba"}
reasoning, code = node.evolve([input_dict], "i1", replace=True)
node(input_dict)

Could not load vllm class, check CUDA support and GPU RAM size



</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [2]:
from methods.meta_prompt import MetaPlan, extract_json_from_text

mp = MetaPlan("Help me date Dilireba, I'm a white colar worker living on salary.")
prompt = mp._get_prompt_i1()
print("MetaPrompt I1: ")
print(prompt)

from methods.llm import get_openai_response
response = get_openai_response(prompt)


MetaPrompt I1: 
Goal: Help me date Dilireba, I'm a white colar worker living on salary.
First, describe the intuition for your tactics and main steps in one sentence. The description must be inside a brace.
Generate a JSON-style plan represented as a Directed Acyclic Graph (DAG) to achieve the goal. Use creative topology in the DAG, include parallel tasks if required.

The plan should include:

- **Nodes**: Each node represents a key action or step and must contain the following attributes:
  - `task`: Description of the task.
  - `name`: Concise name used for the task function.
  - `input`: The resources, information, or prerequisites needed to perform the action.
  - `output`: The immediate result or outcome of the action.
  - `target`: The purpose or goal that the action contributes to.
  - `mode`: The execution mode for this task ("CODE" or "PROMPT").

- **Edges**: Each edge represents a dependency or relationship between nodes, indicating that one step supports or leads to another

In [11]:
import re 
# from methods.meta_prompt import parse_json_from_response

tactic = re.findall(r"\{(.*)\}", response, re.DOTALL)
plan_dict = extract_json_from_text(response)

In [5]:
from methods.meta_prompt import MetaPrompt, MetaPlan, extract_json_from_text
from methods.eoh_evolution import EvolNode
from methods.llm import get_openai_response as get_response
from typing import Optional, List, Dict

# Collect MetaPrompt from parsed Plan-Graph Response
meta_node_prompts = []
for node in plan_dict["nodes"]:
    node_prompt = MetaPrompt(task=node.get("task"),  func_name=node.get("name"), input=node.get("input"), output=node.get("output"), mode=node.get("mode").lower())
    meta_node_prompts.append(node_prompt)
    
edges = plan_dict["edges"]

EvolNode(meta_prompt = node_prompt)


#### EvalNode Test: 
At the beginning level, generation of EvalNode (when code is used here) should compile with success, and it should takes input and produce output. 

In [1]:
from tools.repo import *


def main():
    repo_url = input("Enter the github repo your are interested: ")
    repo_url = "https://github.com/yilundu/ired_code_release"
    temp_repo = "temp_dir"
    clone_repo(repo_url, temp_repo)
    python_files = get_python_files(temp_repo)
    
    print("Available Python files:")
    for i, file in enumerate(python_files):
        print(f"{i+1}. {file}")
    
    choice = int(input("Enter the number of the file you want to analyze: ")) - 1
    start_file = python_files[choice]
    
    print(f"Building dependency graph for {start_file}...")
    graph = build_function_level_mermaid_graph(start_file, temp_repo)
    # graph = build_mermaid_graph(start_file, temp_repo)
    
    print("Visualizing graph...")
    visualize_function_level_graph(graph)

    import shutil
    shutil.rmtree(temp_repo)

if __name__ == "__main__":
    main()

Available Python files:
1. models.py
2. reasoning_dataset.py
3. dataset.py
4. irem_baseline.py
5. gen_planning_dataset.py
6. planning_dataset.py
7. train.py
8. sat_dataset.py
9. diffusion_lib/denoising_diffusion_pytorch_1d.py
10. diffusion_lib/transformer.py
11. diffusion_lib/nlm.py
12. diffusion_lib/nlm_utils.py
13. irem_lib/irem.py
Building dependency graph for models.py...
Visualizing graph...


Function-level dependency graph saved as 'function_level_dependency_graph.png'
Function-level Mermaid graph saved to 'function_level_dependency_graph.mmd'


In [19]:
from tools.repo import *

# Seems to have bug therein ... 
# repo_url = input("Enter the github repo your are interested: ")
repo_url = "https://github.com/yilundu/ired_code_release"
temp_repo = "temp_repo"
if not os.path.exists(temp_repo):
    clone_repo(repo_url, temp_repo)
else:
    print("Repo already cloned, skipping cloning...")

python_files = get_python_files(temp_repo) # get all python files 
start_file = python_files[0] # pick the first one for debugging purpose 

graph = build_cross_file_mermaid_graph(temp_repo, start_file)

# visualize_function_level_graph(graph)

Repo already cloned, skipping cloning...


In [21]:
print(graph)

graph TD
    node1["<b>models.py</b>"]
    style node1 fill:#f9f,stroke:#333,stroke-width:2px
    node2("swish")
    node1 --> node2
    node3["RMSNorm"]
    style node3 fill:#bbf,stroke:#333,stroke-width:2px
    node1 --> node3
    node4("__init__")
    node3 --> node4
    node5("forward")
    node3 --> node5
    node6["Attend"]
    style node6 fill:#bbf,stroke:#333,stroke-width:2px
    node1 --> node6
    node7("__init__")
    node6 --> node7
    node8("forward")
    node6 --> node8
    node9["Attention"]
    style node9 fill:#bbf,stroke:#333,stroke-width:2px
    node1 --> node9
    node10("__init__")
    node9 --> node10
    node10 --> node6
    node10 --> node3
    node11("forward")
    node9 --> node11
    node12["ResBlock"]
    style node12 fill:#bbf,stroke:#333,stroke-width:2px
    node1 --> node12
    node13("__init__")
    node12 --> node13
    node14("forward")
    node12 --> node14
    node14 --> node2
    node15["SinusoidalPosEmb"]
    style node15 fill:#bbf,stroke:#333,str

In [7]:
def clean_mermaid_graph(graph_string):
    lines = graph_string.split('\n')
    cleaned_lines = ['graph TD']
    nodes = {}
    edges = set()
    styles = {}

    def clean_node_name(name):
        # Remove HTML tags
        name = re.sub(r'<[^>]+>', '', name)
        # Replace spaces and special characters
        name = re.sub(r'[^a-zA-Z0-9_]', '_', name)
        return name

    for line in lines:
        line = line.strip()
        
        # Node definitions
        node_match = re.match(r'(node\d+)\[\"(.+?)\"\]', line)
        if node_match:
            node_id, node_name = node_match.groups()
            clean_name = clean_node_name(node_name)
            nodes[node_id] = clean_name
        
        # Edges
        edge_match = re.match(r'(node\d+)\s*(-->|-\.->)\s*(node\d+)', line)
        if edge_match:
            source, edge_type, target = edge_match.groups()
            if source in nodes and target in nodes:
                edges.add(f'    {nodes[source]} {edge_type} {nodes[target]}')
        
        # Styles
        style_match = re.match(r'style\s+(node\d+)\s+(.+)', line)
        if style_match:
            node_id, style = style_match.groups()
            if node_id in nodes:
                styles[nodes[node_id]] = style

    # Add nodes with quotes
    for node_name in nodes.values():
        cleaned_lines.append(f'    {node_name}["{node_name}"]')

    # Add edges
    cleaned_lines.extend(edges)

    # Add styles
    for node_name, style in styles.items():
        cleaned_lines.append(f'    style {node_name} {style}')

    return '\n'.join(cleaned_lines)


clean_graph = clean_mermaid_graph(graph)
visualize_function_level_graph(clean_graph)


Function-level dependency graph saved as 'function_level_dependency_graph.png'
Function-level Mermaid graph saved to 'function_level_dependency_graph.mmd'


In [14]:
print(clean_graph)

graph TD
    models_py["models_py"]
    RMSNorm["RMSNorm"]
    Attend["Attend"]
    Attention["Attention"]
    ResBlock["ResBlock"]
    SinusoidalPosEmb["SinusoidalPosEmb"]
    EBM["EBM"]
    AutoencodeModel["AutoencodeModel"]
    SudokuLatentEBM["SudokuLatentEBM"]
    SudokuEBM["SudokuEBM"]
    SudokuDenoise["SudokuDenoise"]
    SudokuTransformerEBM["SudokuTransformerEBM"]
    GraphEBM["GraphEBM"]
    GraphReverse["GraphReverse"]
    NLMConvBlock["NLMConvBlock"]
    NLMConv1DBlock["NLMConv1DBlock"]
    GNNConvEBM["GNNConvEBM"]
    NLMConv1DBlockV2["NLMConv1DBlockV2"]
    GNNConv1DEBMV2["GNNConv1DEBMV2"]
    GNNConv1DReverse["GNNConv1DReverse"]
    DiffusionWrapper["DiffusionWrapper"]
    GNNDiffusionWrapper["GNNDiffusionWrapper"]
    GNNConvDiffusionWrapper["GNNConvDiffusionWrapper"]
    GNNConv1DV2DiffusionWrapper["GNNConv1DV2DiffusionWrapper"]
    diffusion_lib_nlm_py["diffusion_lib_nlm_py"]
    LogicLayer["LogicLayer"]
    LogicMachine["LogicMachine"]
    diffusion_lib_nlm_utils_py

In [12]:
from tools.repo import _extract_subgraph

subgraph = _extract_subgraph(clean_graph, "Attention")
# visualize_function_level_graph(subgraph)


In [13]:
print(subgraph)

Error: Attention not found in the graph


In [34]:
clean_graph = clean_mermaid_graph(subgraph)
visualize_function_level_graph(clean_graph)

Function-level dependency graph saved as 'function_level_dependency_graph.png'
Function-level Mermaid graph saved to 'function_level_dependency_graph.mmd'


In [22]:
g = """graph TD
    Attention["Attention"]
    Init["__init__"]
    Forward["forward"]
    RMSNorm["RMSNorm"]
    Attend["Attend"]
    File["diffusion_lib/nlm_utils.py"]

    Attention --> Init
    Attention --> Forward
    Attention -.-> RMSNorm
    Attention -.-> Attend
    Init --> RMSNorm
    Init --> Attend

    style Attention fill:#bbf,stroke:#333,stroke-width:2px
    style Init fill:#fff,stroke:#333,stroke-width:1px
    style Forward fill:#fff,stroke:#333,stroke-width:1px
    style RMSNorm fill:#fff,stroke:#333,stroke-width:1px
    style Attend fill:#fff,stroke:#333,stroke-width:1px
    style File fill:#f9f,stroke:#333,stroke-width:2px"""

visualize_function_level_graph(g)

Function-level dependency graph saved as 'function_level_dependency_graph.png'
Function-level Mermaid graph saved to 'function_level_dependency_graph.mmd'


In [15]:
print(graph)

graph TD
    node1["<b>models.py</b>"]
    style node1 fill:#f9f,stroke:#333,stroke-width:2px
    node2("swish")
    node1 --> node2
    node3["RMSNorm"]
    style node3 fill:#bbf,stroke:#333,stroke-width:2px
    node1 --> node3
    node4("__init__")
    node3 --> node4
    node5("forward")
    node3 --> node5
    node6["Attend"]
    style node6 fill:#bbf,stroke:#333,stroke-width:2px
    node1 --> node6
    node7("__init__")
    node6 --> node7
    node8("forward")
    node6 --> node8
    node9["Attention"]
    style node9 fill:#bbf,stroke:#333,stroke-width:2px
    node1 --> node9
    node10("__init__")
    node9 --> node10
    node10 --> node3
    node10 --> node6
    node11("forward")
    node9 --> node11
    node12["ResBlock"]
    style node12 fill:#bbf,stroke:#333,stroke-width:2px
    node1 --> node12
    node13("__init__")
    node12 --> node13
    node14("forward")
    node12 --> node14
    node14 --> node2
    node15["SinusoidalPosEmb"]
    style node15 fill:#bbf,stroke:#333,str

In [15]:
print("Single File Code Structure Parsing: ")

from tools.repo import build_minimal_mermaid_graph, visualize_function_level_graph

file_path = "test.py"
directory = "temp_dir"
graph = build_minimal_mermaid_graph(directory, "test.py")
print(graph)
visualize_function_level_graph(graph)

Single File Code Structure Parsing: 
graph TD
    node1["test.py"]
    node2["test_subfunction"]
    node1 --> node2
    node3["TestClass"]
    node1 --> node3
    node4["__init__"]
    node3 --> node4
    node5["test_method"]
    node3 --> node5
    node5 --> node2


Function-level dependency graph saved as 'function_level_dependency_graph.png'
Function-level Mermaid graph saved to 'function_level_dependency_graph.mmd'


In [16]:
print("Cross-file Code Structure Parsing: ")

from tools.repo import build_cross_file_mermaid_graph, visualize_function_level_graph

file_name = "comp.py"
directory = "temp_dir"

graph = build_cross_file_mermaid_graph(directory, file_name)
# print(graph)

visualize_function_level_graph(graph)

Cross-file Code Structure Parsing: 


Function-level dependency graph saved as 'function_level_dependency_graph.png'
Function-level Mermaid graph saved to 'function_level_dependency_graph.mmd'


In [17]:
from tools.repo import extract_subgraph

# Extract subgraph for TestClass
test_class_subgraph = extract_subgraph(graph, "TestClass")

# Visualize the subgraph
visualize_function_level_graph(test_class_subgraph)

# Print the subgraph string (optional)
# print(test_class_subgraph)

Function-level dependency graph saved as 'function_level_dependency_graph.png'
Function-level Mermaid graph saved to 'function_level_dependency_graph.mmd'


In [18]:
print(graph)

graph TD
    node1["<b>comp.py</b>"]
    style node1 fill:#f9f,stroke:#333,stroke-width:2px
    node2("another_function")
    node1 --> node2
    node3["CompositeClass"]
    style node3 fill:#bbf,stroke:#333,stroke-width:2px
    node1 --> node3
    node4("__init__")
    node3 --> node4
    node5("composite_method")
    node3 --> node5
    node5 --> node2
    node6["<b>test.py</b>"]
    style node6 fill:#f9f,stroke:#333,stroke-width:2px
    node7("test_subfunction")
    node6 --> node7
    node8["TestClass"]
    style node8 fill:#bbf,stroke:#333,stroke-width:2px
    node6 --> node8
    node9("__init__")
    node8 --> node9
    node10("test_method")
    node8 --> node10
    node10 --> node7
    node1 -.-> node6
    node3 -.-> node2
    node3 -.-> node8
    node8 -.-> node7
